# Part 3.1. Convolution

## 1. Convolution
이미지 위에서 `stride` 값 만큼 `filter`를 이동시키면서 겹쳐지는 부분의 각 원소의 값을 곱해서 모두 더한 값을 출력하는 연산

![13-1.png](./img/13-1.png)

### 1.1. Stride and Padding
* `stride` : filter를 한 번에 얼마나 이동할 것인가
* `padding` : zero padding. 1이면 한 겹의 띠가 입력에 감싸진다.

### 1.2. `nn.Conv2d`

![13-2.png](./img/13-2.png)

Convolution Layer를 생성하기 위한 클래스이다.
* `in_channels` : 입력의 채널 개수
* `out_channels` : 필터의 개수
* `kernel_size` : 필터의 크기. n을 입력하면 nxn이지만 특정 필터사이즈 mxn를 원하면 (m, n)을 입력으로 넣어주면 된다.



#### 1.2.1. Convolution의 Input
`torch.Tensor` 자료형이며 `(batch_size, channel, height, width)`로 구성되어 있다. 이를 Layer에 넣어주면 아래와 같은 Output이 나오게 된다.

#### 1.2.2. Convolution의 Output

$$Output Size = \frac{input size - filter size + (2 * padding)}{stride} + 1 $$

* 소수점이 있을 경우 버린다.
* Input의 가로, 세로가 같지 않고 다를 경우 각각의 사이즈를 계산해준다.


### 1.3. Neuron과 Convolution

![13-3.png](./img/13-3.png)

## 2. Pooling
이미지 사이즈를 줄이기 위해 (=조절하기 위해) 하는 연산

![13-4.png](./img/13-4.png)

### 2.1. `nn.MaxPool2d`

![13-5.png](./img/13-5.png)

Max Pooling Layer를 생성하기 위한 클래스이다.


## 3. MNIST Example

### 3.1. 라이브러리 가져오기

In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

### 3.2. GPU와 seed값 설정

In [0]:
# torch.cuda가 사용가능할 때 GPU를 사용. 아니면 CPU 사용.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# seed 설정
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

### 3.3. 하이퍼 파라미터 설정

In [0]:
# 하이퍼 파라미터
learning_rate = 0.001
training_epochs = 15
batch_size = 100

### 3.4. 데이터셋 로드 및 dataloader 생성

In [0]:
# 다운로드 시 Tensor Value가 아님. 그것을 transform에 인자를 주어서 Tensor로 변환
mnist_train = dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)

In [0]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

### 3.5. 학습 모델 설계

![13-6.png](./img/13-6.png)

* `view` : 한 batch에서의 연산 결과를 n차원이 아닌 1차원으로 변환. 첫 번째 인자로 batch의 크기를 넣고 나머지는 -1로 해서 알아서 계산하게 놔둠.
* `fc` : input으로 연산의 개수를, output으로 label의 개수를 넣어 분류한다.

In [0]:
# CNN 모델 (3 Layer)
class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc = nn.Linear(7*7*64, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)

    out = out.view(out.size(0), -1)
    out = self.fc(out)

    return out

In [7]:
model = CNN().to(device)
print(model)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)


![13-7.png](./img/13-7.png)


In [0]:
# 더 깊은 Layer를 가진 CNN 모델 (5 Layer)
class CNN_More_Deep(nn.Module):

  def __init__(self):
    super(CNN_More_Deep, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc1 = nn.Linear(3*3*128, 625)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(625, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)

    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)

    return out

In [9]:
model_deep = CNN_More_Deep().to(device)
print(model_deep)

CNN_More_Deep(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)


### 3.6. Cost Function 및 Optimizer 정의

In [0]:
# CNN 모델
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
# 더 깊은 CNN 모델
criterion_deep = nn.CrossEntropyLoss().to(device)
optimizer_deep = torch.optim.Adam(model_deep.parameters(), lr=learning_rate)

### 3.7. 학습

In [12]:
# CNN 모델 학습
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)    # torch.Tensor -> torch.cuda.Tensor
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("[Epoch {}] cost = {}".format(epoch+1, avg_cost))

print("Learning Finished!")

[Epoch 1] cost = 0.21992921829223633
[Epoch 2] cost = 0.06148950755596161
[Epoch 3] cost = 0.045612744987010956
[Epoch 4] cost = 0.036390554159879684
[Epoch 5] cost = 0.029233038425445557
[Epoch 6] cost = 0.025149565190076828
[Epoch 7] cost = 0.021464256569743156
[Epoch 8] cost = 0.017689982429146767
[Epoch 9] cost = 0.014837917871773243
[Epoch 10] cost = 0.013035827316343784
[Epoch 11] cost = 0.0109112448990345
[Epoch 12] cost = 0.010275710374116898
[Epoch 13] cost = 0.00806193333119154
[Epoch 14] cost = 0.006317066494375467
[Epoch 15] cost = 0.006835271138697863
Learning Finished!


In [13]:
# 더 깊은 CNN 모델 학습
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)    # torch.Tensor -> torch.cuda.Tensor
    Y = Y.to(device)

    optimizer_deep.zero_grad()
    hypothesis = model_deep(X)

    cost = criterion_deep(hypothesis, Y)
    cost.backward()
    optimizer_deep.step()

    avg_cost += cost / total_batch

  print("[Epoch {}] cost = {}".format(epoch+1, avg_cost))

print("Learning Finished!")

[Epoch 1] cost = 0.16298554837703705
[Epoch 2] cost = 0.04253935441374779
[Epoch 3] cost = 0.02870536968111992
[Epoch 4] cost = 0.02103027142584324
[Epoch 5] cost = 0.015723809599876404
[Epoch 6] cost = 0.013301599770784378
[Epoch 7] cost = 0.01161334291100502
[Epoch 8] cost = 0.01146895531564951
[Epoch 9] cost = 0.007956428453326225
[Epoch 10] cost = 0.008535247296094894
[Epoch 11] cost = 0.006353675853461027
[Epoch 12] cost = 0.00805199146270752
[Epoch 13] cost = 0.005329805891960859
[Epoch 14] cost = 0.006823394913226366
[Epoch 15] cost = 0.002981260884553194
Learning Finished!


### 3.8. 성능 확인

In [14]:
# CNN 모델
# 학습을 하지 않으니까 no gradient 설정
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print("Accuracy :", accuracy.item())

Accuracy : 0.9869999885559082


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [15]:
# 더 깊은 CNN 모델
# 학습을 하지 않으니까 no gradient 설정
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model_deep(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print("Accuracy :", accuracy.item())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy : 0.9887999892234802
